In [2]:
import pandas_gbq
import pydata_google_auth

SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
]

credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    # Note, this doesn't work if you're running from a notebook on a
    # remote sever, such as over SSH or with Google Colab. In those cases,
    # install the gcloud command line interface and authenticate with the
    # `gcloud auth application-default login` command and the `--no-browser`
    # option.
    auth_local_webserver=True,
)


def get_query(category):
    df = pandas_gbq.read_gbq(
        '''
    select  mct.cnslt_id, # 상담이력 번호
            ticcm.conversation_id,
            mct.date,
            mct.category_kor,
            mct.chat_conversation_key, # 채팅 Key
            tic.id, # 대화참여자 Key
            ticcm.type,
            case when ticp.purpose = 'AGENT' then '상담사' else '고객' end as purpose,
            case when ticp.purpose = 'AGENT' and ticp.sub_purpose <> 'MAIN' then '지원상담사' else '' end as sub_purpose,
            datetime(ticcm.created_at,'Asia/Seoul') as created_at, # 메시지 입력 시간
            ticcm.message # 메시지
        from    (select  cnslt_id, chat_conversation_key, category, category_kor, date
                    from    `socar-data.socar_biz_base.mart_consultant_table`
                    where   ifnull(chat_conversation_key,'') <> '') as mct # 실제 채팅이 이루어진 채팅상담만 조회
                left outer join
                (select conversation_key, id
                    from    `socar-data.socar_talk.tb_ir_conversation`) as tic # 대화 Key를 이용하여 대화 ID를 가져온다
                on  mct.chat_conversation_key = tic.conversation_key
                left outer join
                (select conversation_id, participant_key, purpose, sub_purpose
                    from    `socar-data.socar_talk.tb_ir_conversation_participant`
                    where   purpose <> 'ACD') as ticp # 시스템 메시지 제외, 대화 ID를 이용하여 대화에 참여한 참가자의 Key 를 가져온다.
                on  tic.id = ticp.conversation_id
                left outer join
                (select created_by, message, created_at, conversation_id, type
                    from `socar-data.socar_talk.tb_ir_conversation_chat_message`   # 대화 ID와 참가자 Key를 이용하여 대화메시지를 가져온다
                    where created_at between '2023-03-01' and '2023-04-30' ) as ticcm
                on  ticp.conversation_id = ticcm.conversation_id
                and ticp.participant_key = ticcm.created_by
        where mct.category_kor = "%s" # 원하는 카테고리로 이름 변경
        and ticcm.conversation_id is not null
        order by cnslt_id, created_at
        '''%(category),
        project_id='socar-data',
        credentials=credentials,
    )
    return df

raw_set = get_query("환불 문의 (& 영수증, 증빙, 반려)")
raw_set.head()

Downloading: 100%|██████████|


,cnslt_id,conversation_id,date,category_kor,chat_conversation_key,id,type,purpose,sub_purpose,created_at,message
0,4380535,740587,2023-03-01,"환불 문의 (& 영수증, 증빙, 반려)",3bab59ac-3e88-478e-8f22-c6094ab77e52,740587,MESSAGE,고객,,2023-03-01 10:26:32,"""결제/환불&gt;환불&gt;택시비"" 로 상담이 신청되었습니다."
1,4380535,740587,2023-03-01,"환불 문의 (& 영수증, 증빙, 반려)",3bab59ac-3e88-478e-8f22-c6094ab77e52,740587,JOIN,상담사,,2023-03-01 10:26:35,정소라 상담사가 채팅에 참여하였습니다.
2,4380535,740587,2023-03-01,"환불 문의 (& 영수증, 증빙, 반려)",3bab59ac-3e88-478e-8f22-c6094ab77e52,740587,GREETINGS,상담사,,2023-03-01 10:26:35,"안녕하세요, 고객님. 쏘카 상담사 정소라 입니다. 무엇을 안내해드릴까요?"
3,4380535,740587,2023-03-01,"환불 문의 (& 영수증, 증빙, 반려)",3bab59ac-3e88-478e-8f22-c6094ab77e52,740587,MESSAGE,고객,,2023-03-01 10:27:43,택시비환불건에대해수차례신청하였는데 계속거절당하네요
4,4380535,740587,2023-03-01,"환불 문의 (& 영수증, 증빙, 반려)",3bab59ac-3e88-478e-8f22-c6094ab77e52,740587,MESSAGE,상담사,,2023-03-01 10:27:52,환불 관련하여 문의주시어 정확한 상담을 위해 정보 확인 후 안내 도와드리도록 하겠습...


In [3]:
only_messages_idx = [idx for idx in range(len(raw_set)) if raw_set.iloc[idx]['type']=="MESSAGE"]
only_messages = raw_set.iloc[only_messages_idx]
only_messages.head()

,cnslt_id,conversation_id,date,category_kor,chat_conversation_key,id,type,purpose,sub_purpose,created_at,message
0,4380535,740587,2023-03-01,"환불 문의 (& 영수증, 증빙, 반려)",3bab59ac-3e88-478e-8f22-c6094ab77e52,740587,MESSAGE,고객,,2023-03-01 10:26:32,"""결제/환불&gt;환불&gt;택시비"" 로 상담이 신청되었습니다."
3,4380535,740587,2023-03-01,"환불 문의 (& 영수증, 증빙, 반려)",3bab59ac-3e88-478e-8f22-c6094ab77e52,740587,MESSAGE,고객,,2023-03-01 10:27:43,택시비환불건에대해수차례신청하였는데 계속거절당하네요
4,4380535,740587,2023-03-01,"환불 문의 (& 영수증, 증빙, 반려)",3bab59ac-3e88-478e-8f22-c6094ab77e52,740587,MESSAGE,상담사,,2023-03-01 10:27:52,환불 관련하여 문의주시어 정확한 상담을 위해 정보 확인 후 안내 도와드리도록 하겠습...
5,4380535,740587,2023-03-01,"환불 문의 (& 영수증, 증빙, 반려)",3bab59ac-3e88-478e-8f22-c6094ab77e52,740587,MESSAGE,고객,,2023-03-01 10:28:02,네
6,4380535,740587,2023-03-01,"환불 문의 (& 영수증, 증빙, 반려)",3bab59ac-3e88-478e-8f22-c6094ab77e52,740587,MESSAGE,상담사,,2023-03-01 10:28:35,소중한 정보 확인 감사합니다.


In [4]:
from tqdm import tqdm
from ast import literal_eval
current_speaker = None

all_logs = []
cnslt_ids = []
conversation = []
current_text = ""
current_id = None
dialogue_dict = {}

for idx in tqdm(range(len(only_messages))):

    instance = only_messages.iloc[idx]
    cnslt_id = instance['cnslt_id']
    type = instance['type']

    if type == "FILE":
        message = literal_eval(instance['message'])[0]['path']
    else:
        message = str(instance['message'])
    speaker = instance['purpose']

    if not message.endswith("신청되었습니다."):
        if cnslt_id == current_id:
            if speaker == current_speaker:
                ### text 추
                current_text += " " + message
            else:
                ### dialogue dict text 업데이트,
                ### conversation에 dialogue dict 추가
                
                dialogue_dict['text'] = current_text
                dialogue_dict['role'] = current_speaker

                conversation.append(dialogue_dict)

                dialogue_dict = {}
                current_text = message
                current_speaker = speaker
                current_text = message



        else:
            dialogue_dict['text'] = current_text
            dialogue_dict['role'] = current_speaker

            conversation.append(dialogue_dict)
            all_logs.append(conversation)
            cnslt_ids.append(current_id)
            current_id = cnslt_id
            conversation = []
            dialogue_dict = {}
            if (type == "MESSAGE" or type == "FILE") and not message.endswith("신청되었습니다."):
                current_text = message
                current_speaker = speaker
            else:
                current_text = ""
                current_speaker = None

100%|██████████| 19158/19158 [00:00<00:00, 23062.70it/s]


In [5]:
import pandas as pd
dst_test_raw = pd.DataFrame({"cnslt_id": cnslt_ids,
                             "dialogue" : all_logs})
dst_test_raw.head()
## Dialogue Format ##


,cnslt_id,dialogue
0,NaN,"[{'text': '', 'role': None}]"
1,4380535.0,"[{'text': '택시비환불건에대해수차례신청하였는데 계속거절당하네요', 'role..."
2,4380537.0,[{'text': '주차비 환불요청드렸는데 아직도 환불처리가 안되서요 요청에는 환불...
3,4380734.0,[{'text': '차량 인수후 출차 주차비 주차업체 계좌이체시 환불가능여부 및 방...
4,4380891.0,"[{'text': '반갑습니다, 문의내용 말씀해주시면 확인 후 안내 도와드리겠습니다..."


In [6]:
### 상담사 + 고객 포맷 For Ontology & Slot Filling ###

conv_ids = []
conversations = []

current_text = ""

for idx in tqdm(range(1,len(dst_test_raw))):
    instance = dst_test_raw.iloc[idx]
    conv_id = int(instance['cnslt_id'])
    dialogues = instance['dialogue']
    
    for utterance in dialogues: 
        speaker = utterance['role']
        utt = utterance['text']
        
        if speaker == "상담사":
            current_text += "상담사: " + utt
        elif speaker == "고객":
            if current_text == "":
                current_text += "고객: " + utt
            else:
                current_text += " \n고객: " + utt
            conv_ids.append(conv_id)
            conversations.append(current_text)
            current_text = ""

100%|██████████| 1017/1017 [00:00<00:00, 37338.67it/s]


In [7]:
dst_for_ont = pd.DataFrame({"Conv_ID": conv_ids,
                            "conversations" : conversations})
dst_for_ont.head()

,Conv_ID,conversations
0,4380535,고객: 택시비환불건에대해수차례신청하였는데 계속거절당하네요
1,4380535,상담사: 환불 관련하여 문의주시어 정확한 상담을 위해 정보 확인 후 안내 도와드리도...
2,4380535,"상담사: 소중한 정보 확인 감사합니다. 많이 궁금하셨을텐데요, 확인 후 안내드리도록..."
3,4380535,상담사: 기다려주셔서 감사합니다. \n고객: ...
4,4380535,상담사: 고객님 환불 된 부분으로 확인되어 확인되어 번거로우시겠지만 재확인 부탁드립...


In [8]:
dst_for_ont.to_csv("../raw_dataset/dst_for_ont_refund.csv",index=False)

OSError: Cannot save file into a non-existent directory: '../raw_dataset'